# Mid_Test MLOps (Train w Convnext tiny, stream with streamlit)

Connect to mlflow (later streamlit with docker)

In [11]:
import mlflow
import subprocess
import sys

def start_mlflow_server():
    process = subprocess.Popen([
        sys.executable, "-m", "mlflow", "server",
        "--host", "0.0.0.0",
        "--port", "5600",
        "--backend-store-uri", "sqlite:///mlflow.db",
        "--default-artifact-root", "./mlruns"
    ])
    return process

print("Starting MLFlow server...")
mlflow_process = start_mlflow_server()
mlflow.set_tracking_uri("http://127.0.0.1:5600")

Starting MLFlow server...


Classic Machine Learning Pipeline

In [2]:
#Import library
import os, math
import torch
from torch import nn, amp
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from torchvision.models import convnext_tiny, ConvNeXt_Tiny_Weights
from sklearn.model_selection import StratifiedShuffleSplit
from tqdm import tqdm
from torchvision.transforms import RandAugment
from pathlib import Path
import copy, optuna
from optuna.pruners import MedianPruner

c:\Users\owen\anaconda3\envs\torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#Path
Base_Path   = r"C:\Users\owen\MLOPSMid"
Train_Path  = os.path.join(Base_Path, "train", "train")
Test_Path    = os.path.join(Base_Path, "test", "test")

OUT_DIR       = "./weights"
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)


In [4]:
#Variable     -- will not be tuned
BATCH_SIZE    = 32
NUM_WORKERS   = 4
VAL_SIZE      = 0.2
SEED          = 42
IMAGE_SIZE    = 224
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]
EPOCHS        = 20
WARMUP_EPOCHS = 4
Device        = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype         = torch.bfloat16 if Device.type == "cuda" else torch.float32 #Efisiensi GPU :v

print(Device)

cuda


In [5]:
#preprocessing function
def build_transforms(image_size: int):
    train_tf = transforms.Compose([
        transforms.RandomResizedCrop(IMAGE_SIZE, scale=(0.7, 1.0)),
        transforms.RandomHorizontalFlip(p=0.5),
        RandAugment(num_ops=2, magnitude=9),
        transforms.ToTensor(),
        transforms.Normalize(IMAGENET_MEAN, IMAGENET_STD),
        transforms.RandomErasing(p=0.25, scale=(0.02, 0.15), value="random")
    ])
    val_tf = transforms.Compose([
        transforms.Resize(int(image_size * 1.15)),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
    ])
    return train_tf, val_tf

#spliting function --> return idx
def index_split(dataset: datasets.ImageFolder, val_size: float):
    y = [dataset.samples[i][1] for i in range(len(dataset.samples))]
    splitter = StratifiedShuffleSplit(n_splits=1, test_size=val_size, random_state=SEED)
    train_idx, val_idx = next(splitter.split(range(len(y)), y))
    return train_idx, val_idx

In [6]:
# Load dataset train
train = Path(Train_Path)

temp = datasets.ImageFolder(str(train), transform=transforms.ToTensor())
class_names = temp.classes
num_classes = len(class_names)
print(f"Jumlah kelas: {num_classes}, beranggotakan: {class_names}")

train_tf, val_tf = build_transforms(IMAGE_SIZE)
full_train_ds = datasets.ImageFolder(str(train), transform=train_tf)
full_val_ds   = datasets.ImageFolder(str(train), transform=val_tf)

# Splitting according to index returned by the function, so far full_train_ds = full_val_ds
train_idx, val_idx = index_split(full_train_ds, VAL_SIZE)
train_ds = Subset(full_train_ds, train_idx) #Taking only the train slices according to train_idx
val_ds   = Subset(full_val_ds,   val_idx) #Taking only the validation slices according to val_idx

train_loader = DataLoader(
    train_ds, batch_size=BATCH_SIZE, shuffle=True,
    num_workers=NUM_WORKERS, pin_memory=True,
    persistent_workers=(NUM_WORKERS > 0)
)
val_loader = DataLoader(
    val_ds, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=True,
    persistent_workers=(NUM_WORKERS > 0)
)

Jumlah kelas: 5, beranggotakan: ['Bacterial Pneumonia', 'Corona Virus Disease', 'Normal', 'Tuberculosis', 'Viral Pneumonia']


In [7]:
# Load dataset test
test = Path(Test_Path)

test_tf = transforms.Compose([
    transforms.Resize(int(IMAGE_SIZE * 1.15)),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])

test_ds = datasets.ImageFolder(str(test), transform=test_tf)

test_loader = DataLoader(
    test_ds, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=True,
    persistent_workers=(NUM_WORKERS > 0)
)

In [8]:
#Model defining
weights = ConvNeXt_Tiny_Weights.IMAGENET1K_V1
model = convnext_tiny(weights=weights)
in_features = model.classifier[2].in_features
model.classifier[2] = nn.Linear(in_features, num_classes)
model.to(Device)

print(model)

ConvNeXt(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
    )
    (1): Sequential(
      (0): CNBlock(
        (block): Sequential(
          (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (1): Permute()
          (2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=96, out_features=384, bias=True)
          (4): GELU(approximate='none')
          (5): Linear(in_features=384, out_features=96, bias=True)
          (6): Permute()
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): CNBlock(
        (block): Sequential(
          (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (1): Permute()
          (2): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=

In [10]:
class WarmupCosineLR(torch.optim.lr_scheduler._LRScheduler):
    """Linear warmup for WARMUP_EPOCHS, then cosine decay to 0 until EPOCHS."""
    def __init__(self, optimizer, warmup_epochs, max_epochs, last_epoch=-1):
        self.warmup_epochs = max(0, int(warmup_epochs))
        self.max_epochs = max_epochs
        super().__init__(optimizer, last_epoch)
    def get_lr(self):
        e = self.last_epoch
        if e < self.warmup_epochs:
            warm = (e + 1) / max(1, self.warmup_epochs)
            return [base_lr * warm for base_lr in self.base_lrs]
        t = (e - self.warmup_epochs) / max(1, self.max_epochs - self.warmup_epochs)
        cosine = 0.5 * (1 + math.cos(math.pi * t))
        return [base_lr * cosine for base_lr in self.base_lrs]

In [11]:
# Accuracy function
def accuracy_from_logits(logits: torch.Tensor, targets: torch.Tensor) -> float:
    preds = logits.argmax(dim=1)
    return (preds == targets).float().mean().item()

# With this function, we could control what each epochs do
def train_one_epoch(model, loader, criterion, optimizer, scaler, device):
    model.train()                                      #Normal train
    loss_sum, acc_sum, n = 0.0, 0.0, 0
    pbar = tqdm(loader, leave=False, desc="Train")
    for step, (x, y) in enumerate(pbar, 1):
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)
        if y.dtype != torch.long:
            y = y.long()
        optimizer.zero_grad(set_to_none=True)          #reset gradient sebelum epoch berikutnya
        with amp.autocast(device_type=device.type, dtype=dtype):           #Penggunaan bf16
            logits = model(x)
            C = logits.size(1)
            loss = criterion(logits, y)                #Loss
        scaler.scale(loss).backward()                  #backward mendapatkan ulang gradient
        scaler.step(optimizer)                         #perubahan pada weight model
        scaler.update()
        acc = accuracy_from_logits(logits, y)          #Accuracy
        loss_sum += loss.item(); acc_sum += acc; n += 1
        pbar.set_postfix(loss=f"{loss_sum/n:.4f}", acc=f"{acc_sum/n:.4f}") #live value progress update
    return loss_sum / max(1, n), acc_sum / max(1, n)

@torch.no_grad()
def validate(model, loader, criterion, device):
    model.eval()                                       #Normal evaluate
    loss_sum, acc_sum, n = 0.0, 0.0, 0
    for x, y in loader:
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)
        if y.dtype != torch.long:
            y = y.long()
        with amp.autocast(device_type=device.type, dtype=dtype):
            logits = model(x)
            loss = criterion(logits, y)                #Loss
        acc = accuracy_from_logits(logits, y)          #Accuracy
        loss_sum += loss.item(); acc_sum += acc; n += 1
    return loss_sum / max(1, n), acc_sum / max(1, n)


In [12]:
base_model_state = copy.deepcopy(model.state_dict())
best_val_acc, best_epoch = 0.0, -1
ckpt_path = os.path.join(OUT_DIR, "ConvNext_Tiny.pt")

def objective(trial: optuna.Trial):
    #Hyperparameter tuning
    lr_suggest  = trial.suggest_float("lr", 1e-5, 8e-4, log=True)
    wd_suggest  = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    ls_suggest  = trial.suggest_float("label_smoothing", 0.0, 0.2)
    warmup_sugg = trial.suggest_int("warmup_epochs", 1, WARMUP_EPOCHS)

    # Model load
    model.load_state_dict(base_model_state)
    model.to(Device)

    local_criterion = torch.nn.CrossEntropyLoss(label_smoothing=ls_suggest)
    local_optimizer = torch.optim.AdamW(model.parameters(), lr=lr_suggest, weight_decay=wd_suggest)
    local_scheduler = WarmupCosineLR(local_optimizer, warmup_epochs=warmup_sugg, max_epochs=EPOCHS)
    local_scaler = amp.GradScaler("cuda" if Device.type == "cuda" else "cpu")

    best_val_acc, best_epoch = 0.0, -1
    ckpt_path_trial = os.path.join(OUT_DIR, f"trial{trial.number}_ConvNext_Tiny.pt")

    print(f"[TRIAL {trial.number}] lr={lr_suggest:.2e} wd={wd_suggest:.1e} ls={ls_suggest:.2f} warmup={warmup_sugg}")
    for epoch in range(1, EPOCHS + 1):
        t0 = time.time()
        # Normal train
        tr_loss, tr_acc = train_one_epoch(model, train_loader, local_criterion, local_optimizer, local_scaler, Device)
        # Normal evaluate
        va_loss, va_acc = validate(model, val_loader, local_criterion, Device)
        # Refresh scheduler
        local_scheduler.step()
        # Used learning rate
        lr_now = local_optimizer.param_groups[0]["lr"]
        dt = time.time() - t0

        print(f"[T{trial.number}] Epoch {epoch:02d}/{EPOCHS} | lr {lr_now:.2e} | "
              f"train {tr_loss:.4f}/{tr_acc:.4f} | val {va_loss:.4f}/{va_acc:.4f} | {dt:.1f}s")

        trial.report(float(va_acc), step=epoch)
        if trial.should_prune():
            print(f"[TRIAL {trial.number}] pruned at epoch {epoch}")
            raise optuna.TrialPruned()

        # Save best checkpoint untuk trial ini
        if va_acc > best_val_acc:
            best_val_acc, best_epoch = va_acc, epoch
            torch.save({
                "epoch": best_epoch,
                "model_state": model.state_dict(),
                "optimizer_state": local_optimizer.state_dict(),
                "val_acc": best_val_acc,
                "class_names": class_names,
                "image_size": IMAGE_SIZE,
                "config": {
                    "LR": lr_suggest, "WEIGHT_DECAY": wd_suggest,
                    "EPOCHS": EPOCHS, "WARMUP_EPOCHS": warmup_sugg,
                    "LABEL_SMOOTH": ls_suggest
                }
            }, ckpt_path_trial)
            print(f"[INFO][T{trial.number}] Saved best to {ckpt_path_trial} (val_acc={best_val_acc:.4f})")

    print(f"[TRIAL {trial.number}] Best val acc {best_val_acc:.4f} at epoch {best_epoch}.")
    return float(best_val_acc)

# bikin study dan jalanin trial
study = optuna.create_study(
    study_name="convnext_opt",
    direction="maximize",
    pruner=MedianPruner(n_startup_trials=4, n_warmup_steps=3)
)
study.optimize(objective, n_trials=3, gc_after_trial=True)

print("=== Best Trial ===")
print("val_acc:", study.best_value)
print("params:", study.best_params)

# copy checkpoint terbaik ke nama default kamu (biar kompatibel dengan pipeline berikutnya)
best_trial_num = study.best_trial.number
best_trial_ckpt = os.path.join(OUT_DIR, f"trial{best_trial_num}_ConvNext_Tiny.pt")
if os.path.isfile(best_trial_ckpt):
    import shutil
    shutil.copyfile(best_trial_ckpt, ckpt_path)
    print(f"[INFO] Copied best checkpoint from trial {best_trial_num} to {ckpt_path}")
else:
    print("[WARN] Best trial checkpoint not found; leaving original path untouched.")


[I 2025-11-14 10:49:03,448] A new study created in memory with name: convnext_opt


[TRIAL 0] lr=2.23e-05 wd=8.8e-06 ls=0.16 warmup=1


[T0] Epoch 01/20 | lr 2.23e-05 | train 1.0114/0.7381 | val 0.8150/0.8508 | 205.9s
[INFO][T0] Saved best to ./weights\trial0_ConvNext_Tiny.pt (val_acc=0.8508)


[T0] Epoch 02/20 | lr 2.21e-05 | train 0.8119/0.8606 | val 0.7977/0.8556 | 164.9s
[INFO][T0] Saved best to ./weights\trial0_ConvNext_Tiny.pt (val_acc=0.8556)


[T0] Epoch 03/20 | lr 2.17e-05 | train 0.7735/0.8804 | val 0.7539/0.8903 | 164.3s
[INFO][T0] Saved best to ./weights\trial0_ConvNext_Tiny.pt (val_acc=0.8903)


...

[T2] Epoch 17/20 | lr 4.05e-06 | train 0.5929/0.9813 | val 0.7042/0.9184 | 34.4s


[T2] Epoch 18/20 | lr 1.82e-06 | train 0.5887/0.9848 | val 0.7077/0.9184 | 34.4s


[T2] Epoch 19/20 | lr 4.59e-07 | train 0.5877/0.9856 | val 0.7029/0.9193 | 34.5s


[I 2025-11-14 11:52:22,036] Trial 2 finished with value: 0.9223927871177071 and parameters: {'lr': 6.03974035063107e-05, 'weight_decay': 0.0003361444320401668, 'label_smoothing': 0.15730112957081632, 'warmup_epochs': 2}. Best is trial 1 with value: 0.9225450785536515.


[T2] Epoch 20/20 | lr 0.00e+00 | train 0.5835/0.9866 | val 0.7046/0.9201 | 34.4s
[TRIAL 2] Best val acc 0.9224 at epoch 14.
=== Best Trial ===
val_acc: 0.9225450785536515
params: {'lr': 0.00023000700364804216, 'weight_decay': 1.2854956073548836e-06, 'label_smoothing': 0.10384556115948224, 'warmup_epochs': 4}
[INFO] Copied best checkpoint from trial 1 to ./weights\ConvNext_Tiny.pt


Store model on Docker

In [14]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

ckpt = torch.load(r"C:\Users\owen\MLOPSMid\weights\ConvNext_Tiny.pt", map_location=Device)
model.load_state_dict(ckpt["model_state"])

@torch.no_grad()
def evaluate_on_test(model, loader, device, class_names):
    model.eval()
    all_preds = []
    all_labels = []

    use_amp = hasattr(torch, "amp") and device.type == "cuda"

    for xb, yb in loader:
        xb = xb.to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)

        if use_amp:
            with amp.autocast(device_type=device.type, dtype=dtype):
                logits = model(xb)
        else:
            logits = model(xb)

        preds = logits.argmax(dim=1)

        all_preds.append(preds.cpu())
        all_labels.append(yb.cpu())

    all_preds = torch.cat(all_preds).numpy()
    all_labels = torch.cat(all_labels).numpy()

    acc = accuracy_score(all_labels, all_preds)
    cm = confusion_matrix(all_labels, all_preds)

    print(f"\nTest accuracy: {acc * 100:.2f}%\n")
    print("Classification report:")
    print(classification_report(all_labels, all_preds, target_names=class_names))

    return cm, all_labels, all_preds

cm, y_true, y_pred = evaluate_on_test(model, test_loader, Device, class_names)

C:\Users\owen\AppData\Local\Temp\ipykernel_46828\2213047478.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(r"C:\Users\owen\MLOPSMid\weights\ConvNext_T


Test accuracy: 91.72%

Classification report:
                      precision    recall  f1-score   support

 Bacterial Pneumonia       0.86      0.78      0.82       401
Corona Virus Disease       0.99      1.00      0.99       406
              Normal       0.97      0.96      0.96       402
        Tuberculosis       1.00      1.00      1.00       406
     Viral Pneumonia       0.77      0.85      0.81       401

            accuracy                           0.92      2016
           macro avg       0.92      0.92      0.92      2016
        weighted avg       0.92      0.92      0.92      2016



Result: Accuracy is 0.9172 which rounded to 0.92 from best model with:

params: {'lr': 0.00023000700364804216,

'weight_decay': 1.2854956073548836e-06,

'label_smoothing': 0.10384556115948224,

'warmup_epochs': 4}

# Store to mlflow

In [9]:
preproc_dir = Path("preprocessing")
preproc_dir.mkdir(exist_ok=True)

train_tf_path = preproc_dir / "train_tf.txt"
with open(train_tf_path, "w", encoding="utf-8") as f:
    f.write(str(train_tf))

mlflow.log_artifact(str(train_tf_path), artifact_path="preprocessing")

In [10]:
import mlflow.pytorch

ckpt = torch.load(r"C:\Users\owen\MLOPSMid\weights\trial1_ConvNext_Tiny.pt", map_location=Device)
model.load_state_dict(ckpt["model_state"])
model.to(Device)
model.eval()

mlflow.pytorch.log_model(
    model,
    artifact_path="pytorch_model",
)


C:\Users\owen\AppData\Local\Temp\ipykernel_13220\1892898339.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(r"C:\Users\owen\MLOPSMid\weights\trial1_Con

# Stream in Streamlit & Containerization with Docker

In [ ]:
%%writefile predict.py
import torch
from torch import nn
import numpy as np
from torchvision import transforms
from PIL import Image
from torch.cuda import amp
from torchvision.models import convnext_tiny

Device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dtype = torch.bfloat16 if Device.type == "cuda" else torch.float32


CKPT_PATH = r"C:\Users\owen\MLOPSMid\weights\ConvNext_Tiny.pt"
IMAGE_SIZE = 224
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD  = [0.229, 0.224, 0.225]

class_names = [
    "Bacterial Pneumonia",
    "Corona Virus Disease",
    "Normal",
    "Tuberculosis",
    "Viral Pneumonia"
]
num_classes = len(class_names)

eval_tf = transforms.Compose([
    transforms.Resize(int(IMAGE_SIZE * 1.15)),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD),
])

model = convnext_tiny(weights=None)
model.classifier[2] = nn.Linear(model.classifier[2].in_features, num_classes)
ckpt = torch.load(CKPT_PATH, map_location=Device)
model.load_state_dict(ckpt["model_state"])
model.to(Device)
model.eval()

@torch.no_grad()
def predict_one(img_pil: Image.Image, model = model):
    img = eval_tf(img_pil).unsqueeze(0).to(Device)

    logits = model(img)

    probs = torch.softmax(logits, dim=1)[0].cpu().numpy()
    pred_idx = int(probs.argmax())
    pred_class = class_names[pred_idx]
    confidence = float(probs[pred_idx])

    return pred_class, confidence, probs

Overwriting predict.py


In [5]:
%%writefile streamlit_app.py
import streamlit as st
from PIL import Image
import pandas as pd
from predict import predict_one, class_names

st.title("Mid-Test MLOps X-ray image dissease classification")
st.write(f"Upload an image, I’ll classify it as either {class_names}")

uploaded_file = st.file_uploader("Upload here (one image only)", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    img = Image.open(uploaded_file).convert("RGB")
    st.image(img, caption="Uploaded!", use_column_width=True)

    if st.button("Predict"):
        with st.spinner("Loading, sabar ya"):
            pred_class, confidence, probs = predict_one(img)

        st.subheader("Prediction result")
        st.write(f"Predicted class: {pred_class}")
        st.write(f"Confidence: {confidence * 100:.2f}%")

Overwriting streamlit_app.py


better run in terminal, no need to rerun

In [ ]:
# docker pull python:3.10-slim

In [ ]:
# docker build -t mlops-mid-app .

In [ ]:
# docker run --rm -p 8501:8501 mlops-mid-app

run this to deploy my program, first u need to login first

In [ ]:
#docker login

In [ ]:
# docker pull owenputra/mlops-mid-app:latest

In [ ]:
# docker run -p 8501:8501 owenputra/mlops-mid-app:latest

please open this once u run:http://localhost:8501